# Stude Terkel Radio Archive Scraper

Downloads program information from [WFMT's archive](https://studsterkel.wfmt.com/explore#t=date) of radio programs hosted by Studs Terkel.

### Import Python tools

In [ ]:
import os
import time
import boto3
import requests
import pathlib
import pandas as pd
import urllib.parse
import urllib.request
from rich.progress import track
from datetime import date
from bs4 import BeautifulSoup
from boto3.s3.transfer import S3Transfer

### Download program list

Fetch the URL

In [ ]:
r = requests.get("https://studsterkel.wfmt.com/explore#t=date")

Parse it

In [ ]:
html = r.text

In [ ]:
soup = BeautifulSoup(html)

Pull all the segemented blocks on the page

In [ ]:
year_list = soup.find_all(class_="prog_year_block")

Loop through them and grab all the URLs, which lead to program pages.

In [ ]:
link_list = []

In [ ]:
for year in year_list:
    a_list = year.find_all("a")
    link_list.extend([a['href'] for a in a_list])

Make sure that list is unique

In [ ]:
unique_links = set(link_list)

Print the total number of URLs

In [ ]:
len(unique_links)

### Scrape program pages

Prepare functions to parse pages

In [ ]:
def parse_meta(e):
    """
    Parse a grid cell of metadata from the bottom of a program page.
    """
    # Get all the p tags
    p_list = e.find_all("p")
    d = {}
    for p in p_list:
        # Split out the bolded text as the label
        label = p.strong.text
        p.strong.extract()
        # Keep the rest as the value
        value = p.text.strip()
        # Add to the dictionary
        d[label] = value
    # Return all dictionaries in this block
    return d

In [ ]:
def url2name(url):
    return pathlib.Path(urllib.parse.urlparse(url).path).stem + ".html"

In [ ]:
def download_html(url):
    name = url2name(url)
    path = pathlib.Path("./html") / name
    if path.exists():
        return path
    else:
        headers = {
            'User-Agent': 'Studs Terkel Radio Archive Scraper (github.com/palewire/studs-terkel-radio-feed/)',
        }
        r = requests.get(f"https://studsterkel.wfmt.com{url}", headers=headers)
        if not r.status_code == 200:
            print(f"Failed with status code {r.status_code}")
            return None
        with open(path, "w") as fh:
            fh.write(r.text)
        time.sleep(0.15)
        return path

Loop through all URLs and scrape each individual page.

In [ ]:
for url in track(list(unique_links)):
    d = download_html(url)

In [ ]:
def scrape_program(path):
    """
    Scrape a program page and return data.
    """
    # Open the page
    with open(path, "r") as fh:
        html = fh.read()
    
    # Parse the HTML
    soup = BeautifulSoup(html)
    
    # Pull out the title
    title = soup.find("h1").text
    
    # Parse out all metadata
    meta = {}
    for e in soup.find(class_="meta_data__section").find_all(class_="col-4"):
        meta.update(parse_meta(e))
    
    # Grab the MP3 URL, if it exists
    media = soup.find(class_="audio_trigger")
    if media:
        mp3_url = media['data-track-url']
    else:
        mp3_url = None
    
    # Grab the synopsis, if it exists
    summary = soup.find(class_="program_synopsis__body")
    if summary:
        synopsis = summary.h2.text
    else:
        synopsis = None
    
    # Return the scraped data
    return dict(
        title=title,
        mp3_url=mp3_url,
        archive_url="/programs/" + path.stem,
        synopsis=synopsis,
        **meta
    )

In [ ]:
html_list = list(pathlib.Path("./html").glob("*.html"))

In [ ]:
program_list = []
for p in track(html_list):
    d = scrape_program(p)
    program_list.append(d)

Convert to a dataframe

In [ ]:
df = pd.DataFrame(program_list).rename(columns={
    "Broadcast Date": "broadcast_date",
    "Physical Format": "physical_format",
    "Digital Format": "digital_format",
    "Ownership": "ownership",
    "Language": "language",
    "Program Sponsor": "program_sponsor",
    "Duration": "duration"
})

Calculate extra columns

In [ ]:
def parse_date(s):
    if pd.isnull(s):
        return None
    try:
        month, day, year = s.split(" ")
    except ValueError:
        return None
    return pd.to_datetime(s)

In [ ]:
df.archive_url = df.archive_url.str.strip()

In [ ]:
df['broadcast_datetime'] = df.broadcast_date.apply(parse_date)

In [ ]:
df['broadcast_year'] = df.broadcast_datetime.dt.year

In [ ]:
df['broadcast_month'] = df.broadcast_datetime.dt.month

In [ ]:
df['broadcast_monthday'] = df.broadcast_datetime.dt.day

In [ ]:
df['has_mp3_url'] = ~pd.isnull(df.mp3_url)

Calculate statistics

In [ ]:
df.has_mp3_url.value_counts()

### Export

In [ ]:
df.sort_values(["broadcast_month", "broadcast_monthday", "broadcast_year"]).to_csv("../data/programs.csv", index=False)

### Download mp3 files

In [17]:
sked = pd.read_csv("../data/schedule.csv")

In [18]:
has_url = sked[~pd.isnull(sked.archive_url)]

In [19]:
assert not has_url.archive_url.duplicated().any()

In [20]:
programs = pd.read_csv("../data/programs.csv")

In [21]:
feed_df = has_url.merge(
    programs,
    on="archive_url",
    how="inner"
)

In [22]:
this_dir = os.path.abspath("")

In [23]:
parent_dir = os.path.dirname(this_dir)

In [24]:
def download_mp3(mp3_url):
    """
    Download the mp3 URL and return the local file path.
    """
    # If there's no URL, there's no path
    if not mp3_url or pd.isnull(mp3_url):
        return None
    
    # Build the local path using the URL
    filename = mp3_url.split("/")[-1].replace("published%2F", "")
    filepath = pathlib.Path("./mp3") / filename
    
    # If the file is already downloaded, we're good to go
    if filepath.exists():
        pass
    # If not, download it
    else:
        print(f"Downloading {mp3_url} to {filepath}")
        opener = urllib.request.build_opener()
        opener.addheaders = [("Referer", "https://studsterkel.wfmt.com/")]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(mp3_url, filepath)
        time.sleep(0.125)

    # Return the path
    return filepath

In [26]:
feed_df['mp3_path'] = feed_df.mp3_url.apply(download_mp3)

In [29]:
feed_df.mp3_path

0      mp3/40605.mp3
1      mp3/56417.mp3
2       mp3/6537.mp3
3      mp3/10904.mp3
4       mp3/2158.mp3
           ...      
209    mp3/13551.mp3
210    mp3/13552.mp3
211    mp3/34390.mp3
212     mp3/2724.mp3
213    mp3/18621.mp3
Name: mp3_path, Length: 214, dtype: object

### Upload mp3 files

In [40]:
def get_s3_client(bucket, region="us-east-1"):
    """
    Returns a transfer client ready to upload files to an s3 bucket.
    Provide the S3 bucket name and region.
    """
    credentials = { 
        'aws_access_key_id': os.getenv('AWS_ACCESS_KEY_ID'),
        'aws_secret_access_key': os.getenv('AWS_ACCESS_KEY_SECRET')
    }
    client = boto3.client('s3', region, **credentials)
    return S3Transfer(client)

In [41]:
bucket = 'studs-terkel-radio-archive-feed'
client = get_s3_client(bucket)

In [42]:
for path in track(list(feed_df.mp3_path)):
    client.upload_file(
        str(path),
        bucket,
        str(path),
        extra_args={'ACL': 'public-read', 'ContentType': 'audio/mpeg'}
    )

Output()

Uploading mp3/40605.mp3 to studs-terkel-radio-archive-feed

Uploading mp3/56417.mp3 to studs-terkel-radio-archive-feed

Uploading mp3/6537.mp3 to studs-terkel-radio-archive-feed

Uploading mp3/10904.mp3 to studs-terkel-radio-archive-feed

...

Uploading mp3/13551.mp3 to studs-terkel-radio-archive-feed

Uploading mp3/13552.mp3 to studs-terkel-radio-archive-feed

Uploading mp3/34390.mp3 to studs-terkel-radio-archive-feed

Uploading mp3/2724.mp3 to studs-terkel-radio-archive-feed

Uploading mp3/18621.mp3 to studs-terkel-radio-archive-feed

In [ ]:
feed_df['feed_url'] = feed_df.mp3_path.apply(lambda x: f'https://studs-terkel-radio-archive-feed.s3.amazonaws.com/{x}')

In [ ]:
feed_df.sort_values(["feed_date"]).to_csv("../data/feed.csv", index=False)